In [1]:
!pip install pymysql
!pip install pymssql

import LibData
import LibUtils
import LibML
import LibFile
import LibEE

import pandas as pd
import numpy as np

jobName = '201216_Train_split1'

# 단계6. 임베딩2 :  원핫 -> 머니니스 -> 가입나이 -> 금리축소 -> 임베딩_상품코드 -> 임베딩_경과기간x현재나이 -> 스케일러
x, y = LibData.GetXYBase(jobName)
x = LibData.GetMoneyness(x)
x = LibData.GetAttainedAge(x)
x = LibData.GetIrReduction(x)
x = LibEE.GetX_Inter_EM_AA(x)
x = LibEE.GetXEmbApplied(3, x, y, ['ProdCode', 'Inter_EM_AA'])
scaler, x = LibUtils.ScaleStandard(x)

colsSelected = ['ElapsedMth', 'Sex', 'Prem', 'PremYr', 'AccumPrem', 'ContAmt', 'StartAge', 'FundAllo03', 'IR06', 'IR09', 'IR12', 'IR14', 'Moneyness', 'AttainedAge', 'ee_ProdCode_00', 'ee_ProdCode_01', 'ee_ProdCode_02']
x = x[colsSelected]

     |████████████████████████████████| 51kB 2.8MB/s 
     |████████████████████████████████| 1.3MB 5.6MB/s 


In [2]:
from sklearn.ensemble import RandomForestRegressor

# 탐색 대상 함수
def ObjFunc(n_estimators, max_features, min_samples_leaf):

  # 모델 정의
    model = RandomForestRegressor(n_estimators = int(n_estimators),
                                  max_features = int(max_features),
                                  min_samples_leaf = int(min_samples_leaf))
    
    scores = LibML.GetPred(x,y,model,2)

    # return negative mae
    return -scores[0]


In [3]:
# bayesian-optimization 라이브러리의 BayesianOptimization 클래스 import
!pip install bayesian-optimization
from bayes_opt import BayesianOptimization
import numpy as np

pbounds = { 'max_features': (14, len(x.columns)),
            'min_samples_leaf': (1, 3),
            'n_estimators': (1500, 2000)
            }

# Bayesian optimization 객체 생성
# f : 탐색 대상 함수, pbounds : hyperparameter 집합
# verbose = 2 항상 출력, verbose = 1 최댓값일 때 출력, verbose = 0 출력 안함
# random_state : Bayesian Optimization 상의 랜덤성이 존재하는 부분을 통제 
bo=BayesianOptimization(f=ObjFunc, pbounds=pbounds, verbose=2, random_state=1)

# 메소드를 이용해 최대화 과정 수행
# init_points :  초기 Random Search 갯수
# n_iter : 반복 횟수 (몇개의 입력값-함숫값 점들을 확인할지! 많을 수록 정확한 값을 얻을 수 있다.)
# acq : Acquisition Function들 중 Expected Improvement(EI) 를 사용
# xi : exploration 강도 (기본값은 0.0)

bo.maximize(init_points=3, n_iter=70, acq='ei', xi=0.02)

# ‘iter’는 반복 회차, ‘target’은 목적 함수의 값, 나머지는 입력값을 나타냅니다. 
# 현재 회차 이전까지 조사된 함숫값들과 비교하여, 현재 회차에 최댓값이 얻어진 경우, 
# bayesian-optimization 라이브러리는 이를 자동으로 다른 색 글자로 표시하는 것을 확인할 수 있습니다

# 찾은 파라미터 값 확인
print(bo.max)

LibUtils.ELoop()

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=983ce6e23251aa76d5a212c1f16bcf97e482eddb594eeed2fbecad366326194e
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization
|   iter    |  target   | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------
|  1        | -138.5    |  15.25    |  2.441    |  1.5e+03  |
|  2        | -135.7    |  14.91    |  1.294    |  1.546e+0 |
|  3        | -136.4    |  14.56    |  1.691    |  1.698e+0 |
|  4        | -135.8    |  14.71    |  1.714    |  1.546e+0 |
|  5        | -136.7    |  17.0     |  1.0      |  1.543e+0 |
|  6        | -137.4    |  17.0     |  1.0      |  1.548e+0 |
|  7        | -136.2    |  14.31    |  1.603    |  1.545e+0 |
|  8        | -136.1    |  14.25    |  1.099    |  1.546e+0 |
|  9        | -135.8    |  15.64    |  1.716    |  

KeyboardInterrupt: ignored